# Overview
- In this demo, we show how to use decode our En-X model (small) to reproduce the results in Table 4.
- We start with **audio-only Whisper** fine-tuned for En-X translation. We first check the **En ASR WER**, and then check the **En-Ru ST BLEU** score. We test the model with **babble noise from MuAViC at 0-SNR**.
- Finally, we test **audio-visual Whisper-Flamingo.** Again, we first check the **En ASR WER**, then the **En-Ru ST BLEU** score. Whisper-Flamingo outperforms audio-only Whisper on both tasks.
- We recommend connecting to the GPU on colab :)
- NOTE: The **default beam size 1** is fast but slightly worse performance than **beam size 15** (which performs better but is much slower due to batch size = 1). For example, the small model takes 1m to decode with beam size 1 and takes 9m to decode with beam size 15.

# Setup
- Install the required packages and download the resources
- This takes ~4 mins
- NOTE: older version of fairseq requires older version of numpy - Colab may need to restart after installing numpy 1.22. Just run the first cell again and the correct version will already be installed.

In [ ]:
# NOTE: older version of fairseq requires older version of numpy - Colab may need to restart after installing numpy 1.22
# Just run this code again and the correct version will already be installed
!pip install numpy==1.22 tensorboard==2.9.1

In [ ]:
# Verify the correct versions are loaded
import numpy; print(numpy.__version__)
import tensorboard; print(tensorboard.__version__)

1.22.0
2.9.1


In [ ]:
# !pip install uv # faster installs compared to pip, but some problems come up

In [ ]:
# MuAViC instructions for fairseq https://github.com/facebookresearch/muavic
# Clone the "muavic" branch of av_hubert's repo
!git clone -b muavic https://github.com/facebookresearch/av_hubert.git
# Set the fairseq version
%cd av_hubert
!git submodule init
!git submodule update
%cd fairseq
!pip install --editable ./
# !uv pip install --system --editable ./

Cloning into 'av_hubert'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 149 (delta 18), reused 22 (delta 14), pack-reused 111
Receiving objects: 100% (149/149), 4.65 MiB | 3.49 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/av_hubert
Submodule 'fairseq' (https://github.com/pytorch/fairseq) registered for path 'fairseq'
Cloning into '/content/av_hubert/fairseq'...
Submodule path 'fairseq': checked out '272c4c5197250997148fb12c0db6306035f166a4'
/content/av_hubert/fairseq
Obtaining file:///content/av_hubert/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install python-speech-features==0.6 # av-hubert
!pip install tiktoken # whisper
!pip install install pytorch-lightning

# !uv pip install --system tiktoken # whisper
# !uv pip install --system pytorch-lightning
# !uv pip install --system torchmetrics

  Preparing metadata (setup.py) ... done
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=3473dcca9d2d1ebef613f86607c14b6dca40a4161a214c99837b41c3436df954
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python-speech-features
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 59.0 MB/s eta 0:00:00


In [ ]:
%cd /content/
!git clone https://github.com/roudimit/whisper-flamingo.git
%cd whisper-flamingo
%pwd

In [ ]:
import sys
sys.path.insert(0, '/content/av_hubert/fairseq')
import os
import json
import argparse
import numpy as np
import torch
from torch import nn
from scipy.io import wavfile
import pandas as pd
import whisper
from tqdm import tqdm
import sacrebleu
from fairseq.scoring.wer import WerScorer, WerScorerConfig
from spec_augment import spec_augment
from utils import (
    load_data,
    WhisperVideoCollatorWithPadding,
)
from utils_batch_samplers import LengthBatchSampler
from whisper_ft_muavic_video import MuavicVideoDataset
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer, seed_everything

INFO:lightning_fabric.utilities.seed:Seed set to 3407


In [ ]:
# verify that we are using the local whisper
print(whisper.__file__)

/content/whisper-flamingo-dev/whisper/__init__.py


In [ ]:
# download data and models
!wget https://data.csail.mit.edu/public-release-sls/whisper-flamingo/muavic.tar.gz
!wget https://data.csail.mit.edu/public-release-sls/whisper-flamingo/noise.tar.gz
!wget https://data.csail.mit.edu/public-release-sls/whisper-flamingo/models/whisper_en-x_small.pt

--2024-05-06 12:25:01--  https://data.csail.mit.edu/public-release-sls/whisper-flamingo/muavic.tar.gz
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.131.233
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.131.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196215206 (187M) [application/x-gzip]
Saving to: ‘muavic.tar.gz’

muavic.tar.gz       100%[===================>] 187.12M  12.8MB/s    in 16s     

2024-05-06 12:25:18 (11.7 MB/s) - ‘muavic.tar.gz’ saved [196215206/196215206]

--2024-05-06 12:25:18--  https://data.csail.mit.edu/public-release-sls/whisper-flamingo/noise.tar.gz
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.131.233
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.131.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 766132 (748K) [application/x-gzip]
Saving to: ‘noise.tar.gz’

noise.tar.gz        100%[===================>] 748.18K   817KB/s    in 0.9s    

2024-0

In [ ]:
!tar -xf muavic.tar.gz
# adjust the noise tsv files with the correct path to the noise
!tar -xf noise.tar.gz
!echo $(pwd)/noise/babble/muavic/babble_all.wav > ./noise/babble/muavic/test.tsv
!echo $(pwd)/noise/babble/lrs3/noise.wav > ./noise/babble/lrs3/test.tsv

# Decode Whisper En-X Small for En ASR

In [ ]:
# general configs
lang = 'en'
model_type = 'small'
# model_type = 'large-v2'

noise_snr = 0
# noise_snr = 1000 # clean

# For En-X models (Table 4), we use babble noise from MuAViC
noise_fn = 'noise/babble/muavic/test.tsv'
# For En only models (Table 3), we use babble noise from LRS3
# noise_fn = 'noise/babble/lrs3/test.tsv'

checkpoint_root = ''

beam_size = 1
# beam_size = 15

fp16 = True if torch.cuda.is_available() else False


In [ ]:
# large model - session crashed
# !wget https://data.csail.mit.edu/public-release-sls/whisper-flamingo/models/whisper_en-x_large.pt

In [ ]:
# asr config
checkpoint = 'whisper_en-x_small.pt' # audio
# checkpoint = 'whisper_en-x_large.pt' # audio
modalities = 'asr'
use_av_hubert_encoder = 0
av_fusion = 'early'
video_model_path=None,
av_hubert_path=None,


In [ ]:
def setup_data():
    checkpoint_path = os.path.join(checkpoint_root, checkpoint)
    SAMPLE_RATE = 16000
    SEED = 3407
    seed_everything(SEED, workers=True)

    audio_transcript_pair_list = load_data(480000, 350, [lang], muavic_root='',
                                           include_audio_lens=True, translate=True if lang != 'en' else False)
    test_dataset =  audio_transcript_pair_list['test']
    test_dataset = [[i[0], i[1].replace('/data/sls/scratch/roudi/datasets/muavic/',
                                        '/content/whisper-flamingo/'),
                                         i[2], i[3]] for i in test_dataset]# fix paths
    tokenizer = whisper.tokenizer.get_tokenizer(multilingual=True, task='transcribe') # NOTE: use transcribe token as hack
    special_token_set = set(tokenizer.special_tokens.values())

    dataset = MuavicVideoDataset(test_dataset,
                                    tokenizer,
                                    SAMPLE_RATE,
                                    model_type,
                                    max_length=None if checkpoint_path else SAMPLE_RATE * 30,
                                    spec_augment="", # no spec augment
                                    noise_prob=1 if noise_snr == 0 else 0, # TODO: implement non-0 snr
                                    noise_fn = noise_fn,
                                    train=False # video center crop, no flip
                                )

    length_sorter = LengthBatchSampler(batch_bins=SAMPLE_RATE * 40 if checkpoint_path and \
                                    beam_size == 1 else 1,
                                shapes=[i[3] for i in test_dataset],
                                sort_in_batch='descending',
                                sort_batch='descending',
                                drop_last=False)

    dataloader = torch.utils.data.DataLoader(dataset,
                        num_workers=4,
                        collate_fn=WhisperVideoCollatorWithPadding(),
                        batch_sampler=length_sorter
                        )
    return tokenizer, special_token_set, dataloader
tokenizer, special_token_set, dataloader = setup_data()

INFO:lightning_fabric.utilities.seed:Seed set to 3407


Removed 0 samples with missing video (before filtering lengths)
Total hours train : 518.8958222222223
en train 299646 292315
Removed 0 samples with missing video (before filtering lengths)
Total hours valid : 1.1514666666666666
en valid 1200 1200
Removed 0 samples with missing video (before filtering lengths)
Total hours test : 0.85056
en test 1321 1321
Total data lengths
292315
1200
1321
Dataloader max length : None
Loaded 1 noise wavs


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
def load_model():
    checkpoint_path = os.path.join(checkpoint_root, checkpoint)
    print("Loading Whisper")
    whisper_model = whisper.load_model(model_type,
                                    video=True if av_fusion == 'separate' else 0,
                                    video_model_path=video_model_path,
                                    av_hubert_path=av_hubert_path,
                                    av_hubert_encoder=use_av_hubert_encoder,
                                    av_fusion=av_fusion,
                                    add_gated_x_attn=1 if av_fusion == 'separate' else 0)

    if checkpoint_path is not None:
        print("Loading checkpoint")
        state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        print(state_dict.keys())
        state_dict = state_dict['state_dict']
        state_dict_updated = {k[6:]: v  for k, v in state_dict.items()} # remove 'model.'
        try: # newer models have learnable scaler init 1
            whisper_model.load_state_dict(state_dict_updated)
        except BaseException as e:
            print(str(e))
            print("Loading weights with strict=False")
            whisper_model.load_state_dict(state_dict_updated, strict=False)

    if fp16 and use_av_hubert_encoder == 1:
        whisper_model.encoder.video_projection_scalar.half()
        whisper_model.encoder.video_model.half()
        model_to_num_layers = {'small': 12, 'medium': 24, 'large-v2': 32}
        if av_fusion == 'separate':
            for i in range(model_to_num_layers[model_type]):
                whisper_model.decoder.blocks[i].attn_gate.data = whisper_model.decoder.blocks[i].attn_gate.half()
                whisper_model.decoder.blocks[i].ff_gate.data = whisper_model.decoder.blocks[i].ff_gate.half()
    return whisper_model
whisper_model = load_model()

Loading Whisper


100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 51.5MiB/s]


Whisper dropout rate : 0.0
Loading checkpoint
odict_keys(['model.encoder.positional_embedding', 'model.encoder.conv1.weight', 'model.encoder.conv1.bias', 'model.encoder.conv2.weight', 'model.encoder.conv2.bias', 'model.encoder.blocks.0.attn.query.weight', 'model.encoder.blocks.0.attn.query.bias', 'model.encoder.blocks.0.attn.key.weight', 'model.encoder.blocks.0.attn.value.weight', 'model.encoder.blocks.0.attn.value.bias', 'model.encoder.blocks.0.attn.out.weight', 'model.encoder.blocks.0.attn.out.bias', 'model.encoder.blocks.0.attn_ln.weight', 'model.encoder.blocks.0.attn_ln.bias', 'model.encoder.blocks.0.mlp.0.weight', 'model.encoder.blocks.0.mlp.0.bias', 'model.encoder.blocks.0.mlp.2.weight', 'model.encoder.blocks.0.mlp.2.bias', 'model.encoder.blocks.0.mlp_ln.weight', 'model.encoder.blocks.0.mlp_ln.bias', 'model.encoder.blocks.1.attn.query.weight', 'model.encoder.blocks.1.attn.query.bias', 'model.encoder.blocks.1.attn.key.weight', 'model.encoder.blocks.1.attn.value.weight', 'model.enc

In [ ]:
def decode_model(num_batches = -1):
    if beam_size == 1: # greedy search
        options = whisper.DecodingOptions(language=lang, fp16=fp16, without_timestamps=True)
    else:
        options = whisper.DecodingOptions(language=lang, fp16=fp16, without_timestamps=True, beam_size=beam_size)
    hypo, refs = [], []
    whisper_model.eval() # AV-HuBERT batch norm and dropout
    for i, b in enumerate(tqdm(dataloader)):
        if fp16:
            input_ids = b["input_ids"].half().cuda()
            video = b["video"].half().cuda()
        else:
            if torch.cuda.is_available():
              input_ids = b["input_ids"].cuda()
              video = b["video"].cuda()
            else:
              input_ids = b["input_ids"]
              video = b["video"]
        labels = b["labels"]
        with torch.no_grad():
            if modalities == "avsr":
                results = whisper_model.decode(input_ids, options, video)
            elif modalities == "asr":
                results = whisper_model.decode(input_ids, options, video, test_a=True)
            else:
                raise NotImplementedError

            for r, l in zip(results, labels):
                hypo.append(r.text)
                print('HYPO: {}'.format(r.text))

                l[l == -100] = tokenizer.eot
                ref = tokenizer.decode([t for t in l if t.item() not in special_token_set])
                refs.append(ref)
                print('REF: {}'.format(ref))
        if i + 1 == num_batches:
            break
    return hypo, refs

In [ ]:
hypo, refs = decode_model(num_batches=-1)

  0%|          | 0/75 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


HYPO: it is my belief that to create these rare game changes which requires normal law at least
REF:  it is my belief that to create these rare game changers requires you to pull off at least
HYPO: i'm here to tell you the story of crazy love the psychological track that dis произised as love one
REF:  i'm here to tell you the story of crazy love a psychological trap disguised as love one
HYPO: and so many of my early memories involved intricate daydreams where i'd longed across borders
REF:  and so many of my early memories involved intricate daydreams where i would walk across borders


  1%|▏         | 1/75 [00:08<10:53,  8.83s/it]

HYPO: my mother was on the line tell me that doctors had found a lump in his stomach terminal cancer
REF:  my mother was on the line to tell me that doctors had found a lump in his stomach terminal cancer
HYPO: and destroy all the bone marrow cancer patients with massive doses of chemo therapy
REF:  you destroy all the bone marrow in the cancer patient with massive doses of chemotherapy
HYPO: about a year after 9 11 researchers examined a group of women who were pregnant when they were exposed to their world
REF:  about a year after 9 11 researchers examined a group of women who were pregnant when they were exposed to the world
HYPO: you're hearing one presentation after another often representing a group of people who try about how
REF:  you're hearing one presentation after another often representing a group of people a tribe about how
HYPO: that's really what i find so intriguing about the reaction that we've had due to the
REF:  and that's really what i find so intriguing about the

  3%|▎         | 2/75 [00:09<05:07,  4.21s/it]

...

 97%|█████████▋| 73/75 [01:16<00:01,  1.34it/s]

HYPO: i tried everything
REF:  i tried everything
HYPO: so let's start there
REF:  so let's start there
HYPO: i made a mistake
REF:  i made a mistake
HYPO: how did that work
REF:  how does that work
HYPO: he's a great guy
REF:  he's a great guy
HYPO: so what do we need
REF:  what do you mean
HYPO: so what do we do
REF:  so what do we do
HYPO: you know that the effect
REF:  who's at the event
HYPO: they're no more of it
REF:  there's no more of it
HYPO: that's what we do
REF:  that's what we do
HYPO: think about that
REF:  think about that
HYPO: they would stop
REF:  it would stop
HYPO: it goes wrong
REF:  it goes wrong
HYPO: i don't know about you
REF:  i don't know about you
HYPO: just think about it
REF:  just think about it
HYPO: but not yet
REF:  well not yet
HYPO: they got scared and left
REF:  they get scared to move
HYPO: deep in life
REF:  make me laugh
HYPO: we all know that
REF:  and we all know this
HYPO: that's my question
REF:  that's my question
HYPO: women are worth it
R

 99%|█████████▊| 74/75 [01:17<00:00,  1.27it/s]

HYPO: well guess what
REF:  well guess what
HYPO: but you know what
REF:  but you know what
HYPO: that is great
REF:  that is great
HYPO: is a fun call
REF:  did you find the gold
HYPO: i had no idea
REF:  had no idea
HYPO: how does it happen
REF:  how does it happen
HYPO: and guess what
REF:  and guess what
HYPO: how are you doing
REF:  how are you
HYPO: i know i did it
REF:  i know i didn't
HYPO: he goes i want to go
REF:  i want to go
HYPO: life is good
REF:  life is good
HYPO: i don't want to
REF:  i didn't want to
HYPO: me k k go away
REF:  make it go away
HYPO: make it go away
REF:  make it go away
HYPO: what's up there
REF:  let's have that
HYPO: what about harriett
REF:  what about polio
HYPO: they got better
REF:  has it gotten better
HYPO: i did nothing
REF:  i know nothing
HYPO: you can try it
REF:  you can try it
HYPO: i don't care
REF:  i don't care
HYPO: could i go there
REF:  could i go there
HYPO: what do they have
REF:  what do they have
HYPO: and then guess what
REF: 

100%|██████████| 75/75 [01:18<00:00,  1.04s/it]


In [ ]:
def get_score():
    if lang != 'en':
        bleu = sacrebleu.corpus_bleu(hypo, [refs]) #NOTE: [ref] not ref
        print("BLEU: %.4f\n" % bleu.score)
    else:
        scorer = WerScorer(
            WerScorerConfig(
                wer_tokenizer="13a",
                wer_remove_punct=True,
                wer_char_level=False,
                wer_lowercase=True
            )
        )
        for h, r in zip(hypo, refs):
            scorer.add_string(ref=r, pred=h)
            wer = scorer.score()
        print("WER : %.4f" % wer)
get_score()

WER : 17.9676


# Decode Whisper En-X Small for En-Ru ST

In [ ]:
lang = 'ru'
tokenizer, special_token_set, dataloader = setup_data()
hypo, refs = decode_model(num_batches=-1)

INFO:lightning_fabric.utilities.seed:Seed set to 3407


Removed 0 samples with missing video (before filtering lengths)
Total hours train : 518.0667377777778
ru train 299646 292140
Removed 0 samples with missing video (before filtering lengths)
Total hours valid : 1.1514666666666666
ru valid 1200 1200
Removed 0 samples with missing video (before filtering lengths)
Total hours test : 0.5640000000000001
ru test 983 983
Total data lengths
292140
1200
983
Dataloader max length : None
Loaded 1 noise wavs


  0%|          | 0/50 [00:00<?, ?it/s]

HYPO: и уничтожить все костные узлы в раке пациентов с массивными радостями пемотерапии
REF:  Костный мозг ракового больного полностью разрушают массивными дозами химиотерапии и затем замещают его несколькими миллионами здоровых клеток костного мозга,
HYPO: И поэтому я думаю, что природа не то, что просто не дотягивается к человечеству, это не млекопитающая.
REF:  Итак, я думаю, что природа — это не что-то нетронутое человечеством, людьми.
HYPO: И он сказал, что я просто так глубоко заботясь о своих клиентах, что я никогда не буду продавать их одним из наших хрупких продуктов.
REF:  Он ответил: «Мне настолько небезразличны мои клиенты, что я никогда не продам им наш второсортный товар».
HYPO: У нас есть идеи о том, как сделать вещи лучше, и я хочу поделиться тремя из них, которые мы взяли в собственном работе.
REF:  У нас есть идеи, как улучшить ситуацию, и я хочу поделиться тремя из нашего опыта.
HYPO: Я стал учеником психологии и посвятил свою работу, чтобы понять человеческую способ

  2%|▏         | 1/50 [00:07<05:45,  7.06s/it]

...

 96%|█████████▌| 48/50 [00:57<00:01,  1.28it/s]

HYPO: Как это было возможно
REF:  Как это возможно?
HYPO: Они хотят выйти
REF:  Они хотят убраться оттуда.
HYPO: Вы получаете несчастье
REF:  Ей не повезло.
HYPO: Я сульса
REF:  мне очень жаль,
HYPO: Большое спасибо
REF:  Спасибо большое,
HYPO: Так как мы делаем это
REF:  Как же это сделать?
HYPO: Они начнут
REF:  то опять пошел,
HYPO: И я плакала
REF:  И плакала.
HYPO: Это не случилось со мной
REF:  Со мной произошло иначе.
HYPO: Большое спасибо
REF:  (Аплодисменты зала)
HYPO: Какой великолепный был этот
REF:  Как вам такое?
HYPO: Это неправильно
REF:  Это неверно.
HYPO: Я не буду спрашивать
REF:  но не тут-то было.
HYPO: Я все попробовал
REF:  Я попробовала всё.
HYPO: Итак, давайте начнем там
REF:  Так давайте же начнем отсюда.
HYPO: Я совершаю ошибку
REF:  Я ошибся»?
HYPO: Как это работает
REF:  А как это работает?»
HYPO: Он хороший мальчик
REF:  Он замечательный парень,
HYPO: Так что нам нужно
REF:  что случилось?
HYPO: Так что мы делаем
REF:  Мы должны считаться с их потребностями

 98%|█████████▊| 49/50 [00:58<00:00,  1.36it/s]

HYPO: Так почему это
REF:  Почему так происходит?
HYPO: Они сделали больше денег
REF:  Я больше заработал.
HYPO: Вы идете, чтобы помочь
REF:  Мы с радостью поможем.
HYPO: И ты знаешь, что
REF:  И знаете,
HYPO: Кто бы я встретил
REF:  Кого я встречу там?
HYPO: И это все
REF:  вот и всё.
HYPO: И это бандит
REF:  И в этом нет ничего хорошего.
HYPO: Большое спасибо
REF:  Спасибо большое,
HYPO: Представьте, что
REF:  И что же вы думаете?
HYPO: Но ты знаешь, что
REF:  Но чтобы вы знали,
HYPO: Это велико
REF:  Это отличное решение
HYPO: Это фантастический звонок
REF:  Нашли ли мы золото?
HYPO: Нет никакой идеи
REF:  В школьном комитете такого даже представить себе не могли.
HYPO: Как это происходит
REF:  (Аплодисменты) Как это происходит?
HYPO: Представьте, что
REF:  И каков результат?
HYPO: Почему ты
REF:  Она спросила: "Как дела?"
HYPO: Я не хочу, чтобы
REF:  Я не хотела.
HYPO: Пусть уходят
REF:  чтобы ее не стало!"
HYPO: Пусть он уходит
REF:  чтобы ее не стало!"
HYPO: Я ничего не делаю
REF

100%|██████████| 50/50 [00:59<00:00,  1.18s/it]


In [ ]:
get_score()

BLEU: 11.8600



# Decode Whisper-Flamingo En-X Small for En ASR

In [ ]:
# Download video models
!wget https://data.csail.mit.edu/public-release-sls/whisper-flamingo/models/whisper-flamingo_en-x_small.pt
!wget https://data.csail.mit.edu/public-release-sls/whisper-flamingo/models/large_noise_pt_noise_ft_433h_only_weights.pt
# !wget https://data.csail.mit.edu/public-release-sls/whisper-flamingo/models/large_noise_pt_noise_ft_433h_no_weights.pt

--2024-05-06 12:29:22--  https://data.csail.mit.edu/public-release-sls/whisper-flamingo/models/whisper-flamingo_en-x_small.pt
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.131.233
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.131.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2611351155 (2.4G)
Saving to: ‘whisper-flamingo_en-x_small.pt’

whisper-flamingo_en  41%[=======>            ]   1.01G  13.2MB/s    in 81s     

2024-05-06 12:30:44 (12.8 MB/s) - Connection closed at byte 1083227248. Retrying.

--2024-05-06 12:30:45--  (try: 2)  https://data.csail.mit.edu/public-release-sls/whisper-flamingo/models/whisper-flamingo_en-x_small.pt
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.131.233|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 2611351155 (2.4G), 1528123907 (1.4G) remaining
Saving to: ‘whisper-flamingo_en-x_small.pt’

whisper-flamingo_en  82%[++++++++=======>    ]   2.0

In [ ]:
# avsr config
checkpoint= 'whisper-flamingo_en-x_small.pt'
modalities = 'avsr'
use_av_hubert_encoder = 1
av_fusion = 'separate'
video_model_path = 'large_noise_pt_noise_ft_433h_only_weights.pt'
# video_model_path = 'large_noise_pt_noise_ft_433h_no_weights.pt'
av_hubert_path =  '/content/av_hubert/avhubert'

In [ ]:
lang = 'en'
tokenizer, special_token_set, dataloader = setup_data()
whisper_model = load_model()
hypo, refs = decode_model(num_batches=-1)

INFO:lightning_fabric.utilities.seed:Seed set to 3407


Removed 0 samples with missing video (before filtering lengths)
Total hours train : 518.8958222222223
en train 299646 292315
Removed 0 samples with missing video (before filtering lengths)
Total hours valid : 1.1514666666666666
en valid 1200 1200
Removed 0 samples with missing video (before filtering lengths)
Total hours test : 0.85056
en test 1321 1321
Total data lengths
292315
1200
1321
Dataloader max length : None
Loaded 1 noise wavs
Loading Whisper
Whisper dropout rate : 0.0
Loading AV-HuBERT encoder


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Using AV-HuBERT encoder with parameters: 325136104
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Loading checkpoint
odict_keys(['model.encoder.video_projection_scalar', 'model.encoder.positional_embedding', 'model.encoder.conv1.weight', 'model.encoder.conv1.bias', 'model.encoder.conv2.weight', 'model.encoder.conv2.bias', 'model.encoder.blocks.0.attn.query.weight', 'model.encoder.blocks.0.attn.query.bias', 'model.encoder.blocks.0.attn.key.weight', 'model.encoder.blocks.0.attn.value.weight', 'model.encoder.blocks.0.attn.value.bias', 'model.encoder.blocks.0.attn.out.weight', 'model.encoder.blocks.0.attn.out.bias', 'model.encoder.blocks.0.attn_ln.weight', 'model.encoder.blocks.0.attn_ln.bias', 'model.encoder.blocks.0

  0%|          | 0/75 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

 97%|█████████▋| 73/75 [01:39<00:01,  1.13it/s]

HYPO: i tried everything
REF:  i tried everything
HYPO: so let's start there
REF:  so let's start there
HYPO: i made a mistake
REF:  i made a mistake
HYPO: how did that work
REF:  how does that work
HYPO: he's a great guy
REF:  he's a great guy
HYPO: so what do you mean
REF:  what do you mean
HYPO: so what do we do
REF:  so what do we do
HYPO: you know that idea right
REF:  who's at the event
HYPO: there's no more of it
REF:  there's no more of it
HYPO: that's what we do
REF:  that's what we do
HYPO: think about that
REF:  think about that
HYPO: it would stop
REF:  it would stop
HYPO: it goes wrong
REF:  it goes wrong
HYPO: i don't know about you
REF:  i don't know about you
HYPO: just think about it
REF:  just think about it
HYPO: well it's not yet
REF:  well not yet
HYPO: they got scared the more
REF:  they get scared to move
HYPO: make me laugh
REF:  make me laugh
HYPO: and we all know this
REF:  and we all know this
HYPO: that's my question
REF:  that's my question
HYPO: women are 

 99%|█████████▊| 74/75 [01:40<00:00,  1.08it/s]

HYPO: well guess what
REF:  well guess what
HYPO: but you know what
REF:  but you know what
HYPO: that is great
REF:  that is great
HYPO: it's a fun call
REF:  did you find the gold
HYPO: i had no idea
REF:  had no idea
HYPO: how does it happen
REF:  how does it happen
HYPO: and guess what
REF:  and guess what
HYPO: how are you
REF:  how are you
HYPO: i know i did it
REF:  i know i didn't
HYPO: he goes i want to go
REF:  i want to go
HYPO: life is good
REF:  life is good
HYPO: i don't want to
REF:  i didn't want to
HYPO: make a goatee
REF:  make it go away
HYPO: make it go away
REF:  make it go away
HYPO: let's not that
REF:  let's have that
HYPO: what about polio
REF:  what about polio
HYPO: they got better
REF:  has it gotten better
HYPO: you know nothing
REF:  i know nothing
HYPO: you can try it
REF:  you can try it
HYPO: i don't care
REF:  i don't care
HYPO: could i go there
REF:  could i go there
HYPO: what do they have
REF:  what do they have
HYPO: and then guess what
REF:  and t

100%|██████████| 75/75 [01:42<00:00,  1.36s/it]


In [ ]:
get_score()

WER : 11.2235


# Decode Whisper-Flamingo En-X Small for En-Ru ST

In [ ]:
lang = 'ru'
tokenizer, special_token_set, dataloader = setup_data()
hypo, refs = decode_model(num_batches=-1)

INFO:lightning_fabric.utilities.seed:Seed set to 3407


Removed 0 samples with missing video (before filtering lengths)
Total hours train : 518.0667377777778
ru train 299646 292140
Removed 0 samples with missing video (before filtering lengths)
Total hours valid : 1.1514666666666666
ru valid 1200 1200
Removed 0 samples with missing video (before filtering lengths)
Total hours test : 0.5640000000000001
ru test 983 983
Total data lengths
292140
1200
983
Dataloader max length : None
Loaded 1 noise wavs


  0%|          | 0/50 [00:00<?, ?it/s]

HYPO: уничтожить все костные узкие в раковом пациенте с массивными радостями химиотерапии
REF:  Костный мозг ракового больного полностью разрушают массивными дозами химиотерапии и затем замещают его несколькими миллионами здоровых клеток костного мозга,
HYPO: И поэтому я думаю, что природа не то, что не связано с человечеством или человечеством.
REF:  Итак, я думаю, что природа — это не что-то нетронутое человечеством, людьми.
HYPO: И он сказал, что я просто так глубоко заботясь о своих клиентах, что я никогда не буду продавать их одним из наших крапивных продуктов.
REF:  Он ответил: «Мне настолько небезразличны мои клиенты, что я никогда не продам им наш второсортный товар».
HYPO: У нас есть идеи о том, как сделать вещи лучше, и я хочу поделиться тремя из них, которые мы взяли в нашей собственной работе.
REF:  У нас есть идеи, как улучшить ситуацию, и я хочу поделиться тремя из нашего опыта.
HYPO: Я стал ученым психологом и посвятил свою работу пониманию человеческой способности забот

  2%|▏         | 1/50 [00:07<06:15,  7.66s/it]

...

 96%|█████████▌| 48/50 [01:13<00:02,  1.00s/it]

HYPO: Скажите, что это возможно
REF:  Как это возможно?
HYPO: Они хотят выйти
REF:  Они хотят убраться оттуда.
HYPO: Вы получаете несчастье
REF:  Ей не повезло.
HYPO: Я так печален
REF:  мне очень жаль,
HYPO: Спасибо вам очень много
REF:  Спасибо большое,
HYPO: Так как мы делаем это
REF:  Как же это сделать?
HYPO: Это началось бы
REF:  то опять пошел,
HYPO: И я плакал
REF:  И плакала.
HYPO: Это не случилось со мной
REF:  Со мной произошло иначе.
HYPO: Большое спасибо
REF:  (Аплодисменты зала)
HYPO: Какой великолепный был этот
REF:  Как вам такое?
HYPO: Это неправильно
REF:  Это неверно.
HYPO: Я не буду спрашивать
REF:  но не тут-то было.
HYPO: Я все попробовал
REF:  Я попробовала всё.
HYPO: Так что давайте начнем там
REF:  Так давайте же начнем отсюда.
HYPO: Я совершил ошибку
REF:  Я ошибся»?
HYPO: Как это работает
REF:  А как это работает?»
HYPO: Он хороший человек
REF:  Он замечательный парень,
HYPO: Так что вы имеете в виду
REF:  что случилось?
HYPO: Так что мы делаем
REF:  Мы должн

 98%|█████████▊| 49/50 [01:14<00:01,  1.01s/it]

HYPO: Так почему это
REF:  Почему так происходит?
HYPO: Я сделал больше денег
REF:  Я больше заработал.
HYPO: Мы идем, чтобы помочь
REF:  Мы с радостью поможем.
HYPO: И ты знаешь, что
REF:  И знаете,
HYPO: Кто бы я встретил
REF:  Кого я встречу там?
HYPO: И это все
REF:  вот и всё.
HYPO: И это бандит
REF:  И в этом нет ничего хорошего.
HYPO: Большое спасибо
REF:  Спасибо большое,
HYPO: Представьте, что
REF:  И что же вы думаете?
HYPO: Но ты знаешь, что
REF:  Но чтобы вы знали,
HYPO: Это великолепно
REF:  Это отличное решение
HYPO: Это веселый звонок
REF:  Нашли ли мы золото?
HYPO: Я не знаю
REF:  В школьном комитете такого даже представить себе не могли.
HYPO: Как это происходит
REF:  (Аплодисменты) Как это происходит?
HYPO: Предположим, что
REF:  И каков результат?
HYPO: Как ты
REF:  Она спросила: "Как дела?"
HYPO: Я не хочу, чтобы
REF:  Я не хотела.
HYPO: Микако уходит
REF:  чтобы ее не стало!"
HYPO: Пусть он уходит
REF:  чтобы ее не стало!"
HYPO: И ты ничего не знаешь
REF:  Я ничего

100%|██████████| 50/50 [01:16<00:00,  1.53s/it]


In [ ]:
get_score()

BLEU: 12.8231

